In [23]:
import csv
import numpy as np
import sys
import pandas as pd
import itertools
import math
import time

from sklearn import svm, linear_model, neighbors
from sklearn import tree, ensemble
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

import networkx as nx
import random
import numbers

from sklearn.model_selection import StratifiedKFold

from src import ml

In [2]:
import findspark
findspark.init("C:/Users/admin-karim/Downloads/spark-2.2.0-bin-hadoop2.7")

from pyspark import SparkConf, SparkContext

In [3]:
if False: 
    sc.stop()

config = SparkConf()
config.setMaster("local[5]")
config.set("spark.executor.memory", "8g")
config.set('spark.driver.memory', '12g')
config.set("spark.memory.offHeap.enabled",True)
config.set("spark.memory.offHeap.size","20g") 
sc = SparkContext(conf=config)
print (sc)

<SparkContext master=local[5] appName=pyspark-shell>


In [12]:
def ddiDataGenerator(ddi_list, vectors):
    ddi_df = pd.read_csv(ddi_list, sep='\t')

    embedding_df = pd.read_csv(vectors, delimiter = '\t') 

    embedding_df.Entity = embedding_df.Entity.str[-8:-1]
    embedding_df.rename(columns = {'Entity':'Drug'}, inplace = True)

    len(set(ddi_df.Drug1.unique()).union(ddi_df.Drug2.unique()) )
    pairs, classes = ml.generatePairs(ddi_df, embedding_df)
    
    return pairs, classes, embedding_df

# Drugbank v5  RDF2Vec SG 

In [17]:
ddi_list = 'input/full_DDI.txt'
vectors = 'vectors/RDF2Vec_cbow_300_5_5_2_500_d5_uniform.txt'

ddi_df = pd.read_csv(ddi_list, sep='\t')
ddi_df.head()

,Drug1,Drug2
0,DB04868,DB09199
1,DB01623,DB00804
2,DB01021,DB08931
3,DB01208,DB09321
4,DB06754,DB07477


In [18]:
pairs, classes, embedding_df = ddiDataGenerator(ddi_list, vectors)

In [19]:
print(pairs)

[['DB00005' 'DB00007']
 ['DB00005' 'DB00009']
 ['DB00005' 'DB00014']
 ...
 ['DB08811' 'DB08816']
 ['DB08811' 'DB08819']
 ['DB08816' 'DB08819']]


In [20]:
print(classes)

[0 0 0 ... 0 0 0]


In [21]:
print(embedding_df.head())

      Drug  feature0  feature1  feature2  feature3  feature4  feature5  \
0  DB00001 -0.903437 -0.331749 -0.369652  0.439574  1.541132 -1.336791   
1  DB08860  1.384768  0.363517  0.557544  2.626180  2.502104  1.148705   
2  DB06605 -0.113727 -0.203172  1.686063  2.237446  1.583634  0.420009   
3  DB06695  1.553845 -1.943734  0.040946  0.987024 -1.168784  0.878595   
4  DB01254  1.605805 -0.298084  0.441323  2.509530  2.398917 -1.055412   

   feature6  feature7  feature8     ...      feature290  feature291  \
0 -1.635298 -0.560466  0.000102     ...       -0.694317   -0.008376   
1 -3.365903 -1.807856 -1.510181     ...        1.365525   -0.723049   
2 -1.796523 -1.203906 -2.252901     ...        0.673704   -0.146191   
3 -0.538334  0.557914 -0.010863     ...        0.997751   -0.345845   
4  0.433553 -2.312896 -0.097910     ...        2.142729   -0.245621   

   feature292  feature293  feature294  feature295  feature296  feature297  \
0    0.411577    0.432441    0.777116    0.081497  

In [25]:
KNN = KNeighborsClassifier(n_neighbors = 3)
NB = GaussianNB()
SVM = svm.SVC()
LR = linear_model.LogisticRegression(C = 0.01)
RF = ensemble.RandomForestClassifier(n_estimators = 5, n_jobs = -1)
GBT = ensemble.GradientBoostingClassifier(n_estimators = 5, max_leaf_nodes = 3, max_depth = 3)

clfs = [('LR',LR),('SVM',SVM),('NB',NB),('KNN',KNN),('GBT',GBT),('RF',LR)]

In [ ]:
n_seed =100
n_fold =5
n_run =2
n_proportion = 1
scoreDF  = ml.kfoldCV(sc, pairs, classes, embedding_df, clfs, n_run, n_fold, n_proportion, n_seed)

In [52]:
scoreDF .groupby(['method','run']).mean().groupby('method').mean()

,accuracy,average_precision,f1,fold,precision,recall,roc_auc
method,,,,,,,
Logistic Regression,0.708657,0.777289,0.708382,2.0,0.709047,0.707720,0.778547
Naive Bayes,0.649174,0.681024,0.633555,2.0,0.663073,0.606556,0.697719
Random Forest,0.837881,0.912630,0.842488,2.0,0.819206,0.867133,0.919453


In [53]:
scoreDF .to_csv('C:/Users/admin-karim/Downloads/GraphEmbedding4DDI-master/GraphEmbedding4DDI-master/Results/results/traditional/DB5/DBv5_TCV_run_cbow.csv',sep=',', index=False)